In this notebook we will optimize the parameters of the rating system

## Imports

In [1]:
from scraping import ufcstats
from glicko2 import Fighter, FighterManager
import pandas as pd
from datetime import datetime

## Data

In [2]:
fights = ufcstats.get_completed_fights(latest_n_events=None)

In [3]:
fights_df = pd.DataFrame(fights).sort_values('date')
fights_df.head()

,event,date,fighter,opponent,weight_class,outcome,method
8057,UFC 2: No Way Out,1994-03-11,Scott Morris,Sean Daugherty,Open Weight,win,SUB
8043,UFC 2: No Way Out,1994-03-11,Royce Gracie,Patrick Smith,Open Weight,win,KO/TKO
8044,UFC 2: No Way Out,1994-03-11,Royce Gracie,Remco Pardoel,Open Weight,win,SUB
8045,UFC 2: No Way Out,1994-03-11,Patrick Smith,Johnny Rhodes,Open Weight,win,SUB
8046,UFC 2: No Way Out,1994-03-11,Royce Gracie,Jason DeLucia,Open Weight,win,SUB


According to [Wikipedia](https://en.wikipedia.org/wiki/Ultimate_Fighting_Championship), UFC 28 (November 17, 2000) was the first UFC event under the "Unified Rules of MMA". This is the current (albeit with some changes) ruleset used by the UFC today (2025). For the purposes of this project, we will only consider fights taking place on or after November 17, 2000.

In [4]:
fights_df = fights_df[fights_df['date'] >= datetime(2000, 11, 17)]
fights_df.head()

,event,date,fighter,opponent,weight_class,outcome,method
7798,UFC 28: High Stakes,2000-11-17,Randy Couture,Kevin Randleman,Heavyweight,win,KO/TKO
7800,UFC 28: High Stakes,2000-11-17,Josh Barnett,Gan McGee,Super Heavyweight,win,KO/TKO
7799,UFC 28: High Stakes,2000-11-17,Renato Sobral,Maurice Smith,Heavyweight,win,M-DEC
7802,UFC 28: High Stakes,2000-11-17,Jens Pulver,John Lewis,Lightweight,win,KO/TKO
7804,UFC 28: High Stakes,2000-11-17,Ben Earwood,Chris Lytle,Welterweight,win,U-DEC


Further we will treat fights graded "No Contest" as if the fight never occured (and thus exclude them).

In [5]:
fights_df = fights_df[fights_df['outcome'] != 'nc']

The following duplicates the entire dataframe, swapping 'fighter' and 'opponent' values and changing the 'outcome' so that there are two rows associated with each fight (i.e. one winner and one loser).

In [6]:
print('Shape before:', fights_df.shape)

copy_df = fights_df.copy()
copy_df[['fighter', 'opponent']] = copy_df[['opponent', 'fighter']]
copy_df['outcome'] = copy_df['outcome'].replace('win', 'loss')
fights_df = pd.concat([fights_df, copy_df], ignore_index=True)

print('Shape after:', fights_df.shape)

Shape before: (7720, 7)
Shape after: (15440, 7)


In [7]:
weekly_grouped = fights_df.groupby(pd.Grouper(key='date', freq='W'))

In [10]:
manager = FighterManager()

for week, group in weekly_grouped:
    manager.update_fighters(group)

In [11]:
sorted([(key, value.rating) for key, value in manager.items()], key=lambda x: x[1], reverse=True)

[('Jon Jones', np.float64(2688.2241459845986)),
 ('Islam Makhachev', np.float64(2575.2356940995596)),
 ('Khabib Nurmagomedov', np.float64(2538.9475856097492)),
 ('Georges St-Pierre', np.float64(2519.2059691718828)),
 ('Ilia Topuria', np.float64(2492.702208620523)),
 ('Shavkat Rakhmonov', np.float64(2475.147074483633)),
 ('Francis Ngannou', np.float64(2439.1846183031744)),
 ('Dricus Du Plessis', np.float64(2438.5738257289927)),
 ('Merab Dvalishvili', np.float64(2420.411340242225)),
 ('Leon Edwards', np.float64(2416.5423665649923)),
 ('Alex Pereira', np.float64(2415.987852679632)),
 ('Belal Muhammad', np.float64(2409.4698253569513)),
 ('Khamzat Chimaev', np.float64(2385.006759723345)),
 ('Daniel Cormier', np.float64(2381.2100098682868)),
 ('Max Holloway', np.float64(2358.0966612931393)),
 ('Stipe Miocic', np.float64(2355.109038749387)),
 ('Movsar Evloev', np.float64(2350.053573122591)),
 ('Ciryl Gane', np.float64(2342.974604537483)),
 ('Alexander Volkanovski', np.float64(2329.87379950876

In [12]:
sorted([(key, value.peak_rating) for key, value in manager.items()], key=lambda x: x[1], reverse=True)

[('Jon Jones', np.float64(2688.2241459845986)),
 ('Islam Makhachev', np.float64(2575.2356940995596)),
 ('Anderson Silva', np.float64(2565.0929269760773)),
 ('Daniel Cormier', np.float64(2551.882260480726)),
 ('Kamaru Usman', np.float64(2550.1912225045007)),
 ('Khabib Nurmagomedov', np.float64(2538.9475856097492)),
 ('Leon Edwards', np.float64(2535.4152996228872)),
 ('Georges St-Pierre', np.float64(2519.2059691718828)),
 ('Stipe Miocic', np.float64(2513.553861860758)),
 ('Ilia Topuria', np.float64(2492.702208620523)),
 ('Chris Weidman', np.float64(2481.766415058375)),
 ('Alexander Volkanovski', np.float64(2481.347763364516)),
 ('Shavkat Rakhmonov', np.float64(2475.147074483633)),
 ('Jiri Prochazka', np.float64(2453.0587406417035)),
 ('Francis Ngannou', np.float64(2439.1846183031744)),
 ('Dricus Du Plessis', np.float64(2438.5738257289927)),
 ('Yoel Romero', np.float64(2422.2537168272966)),
 ('Max Holloway', np.float64(2421.5063884870547)),
 ('Merab Dvalishvili', np.float64(2420.411340242